In [ ]:
import numpy as np
import struct #using struct to unpacj binary data from mnist dataset

#22k-4501
def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        _, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols, 1) / 255.0
    return images

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        _, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels
train_images = load_mnist_images("/kaggle/input/mnist-dataset/train-images.idx3-ubyte")
train_labels = load_mnist_labels("/kaggle/input/mnist-dataset/train-labels.idx1-ubyte")
test_images = load_mnist_images("/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte")
test_labels = load_mnist_labels("/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte")
print(f"Training Data: {train_images.shape}, Labels: {train_labels.shape}")
print(f"Testing Data: {test_images.shape}, Labels: {test_labels.shape}")
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Avoid overflow
    return exp_x / exp_x.sum(axis=-1, keepdims=True)
def cross_entropy_loss(predictions, label):
    return -np.log(predictions[label] + 1e-9)  
class ConvLayer:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = np.random.randn(num_filters, filter_size, filter_size) * 0.1

    def forward(self, input):
        self.last_input = input
        h, w, _ = input.shape
        output = np.zeros((h - self.filter_size + 1, w - self.filter_size + 1, self.num_filters))

        for i in range(output.shape[0]):
            for j in range(output.shape[1]):
                region = input[i:i+self.filter_size, j:j+self.filter_size, 0]
                for f in range(self.num_filters):
                    output[i, j, f] = np.sum(region * self.filters[f])
        
        return np.maximum(output, 0)  # ReLU Activation

    def backward(self, d_out, lr):
        d_filters = np.zeros_like(self.filters)

        for i in range(d_out.shape[0]):
            for j in range(d_out.shape[1]):
                region = self.last_input[i:i+self.filter_size, j:j+self.filter_size, 0]
                for f in range(self.num_filters):
                    d_filters[f] += region * d_out[i, j, f]
        self.filters -= lr * d_filters  
        return d_out 
        
class MaxPoolLayer:
    def __init__(self, size):
        self.size = size

    def forward(self, input):
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros((h // self.size, w // self.size, num_filters))
        self.max_indices = np.zeros_like(input, dtype=bool)

        for i in range(output.shape[0]):
            for j in range(output.shape[1]):
                region = input[i*self.size:(i+1)*self.size, j*self.size:(j+1)*self.size]
                max_value = np.max(region, axis=(0, 1))
                output[i, j] = max_value
                self.max_indices[i*self.size:(i+1)*self.size, j*self.size:(j+1)*self.size] = (region == max_value)

        return output

    def backward(self, d_out):
        d_input = np.zeros_like(self.last_input)
        for i in range(d_out.shape[0]):
            for j in range(d_out.shape[1]):
                d_input[i*self.size:(i+1)*self.size, j*self.size:(j+1)*self.size] = d_out[i, j] * self.max_indices[i*self.size:(i+1)*self.size, j*self.size:(j+1)*self.size]
        return d_input
class FullyConnected:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.1
        self.biases = np.zeros(output_size)

    def forward(self, input):
        self.last_input = input.flatten()
        return softmax(np.dot(self.last_input, self.weights) + self.biases)

    def backward(self, d_out, lr):
        d_w = np.outer(self.last_input, d_out)
        d_b = d_out
        d_input = np.dot(d_out, self.weights.T)

        self.weights -= lr * d_w
        self.biases -= lr * d_b

        return d_input.reshape(self.last_input.shape)
class CNN:
    def __init__(self):
        self.conv1 = ConvLayer(8, 3)
        self.conv2 = ConvLayer(8, 3)
        self.pool = MaxPoolLayer(2)
        sample_input = np.zeros((28, 28, 1))
        x = self.conv1.forward(sample_input)
        x = self.conv2.forward(x)
        x = self.pool.forward(x)
        self.flattened_shape = x.shape 
        flattened_size = np.prod(self.flattened_shape)

        self.fc = FullyConnected(flattened_size, 10)

    def forward(self, x):
        x = self.conv1.forward(x)
        x = self.conv2.forward(x)
        x = self.pool.forward(x)
        return self.fc.forward(x)

    def backward(self, d_out, lr=0.005):
        d_out = self.fc.backward(d_out, lr)
        d_out = d_out.reshape(self.flattened_shape)
        d_out = self.pool.backward(d_out)
        d_out = self.conv2.backward(d_out, lr)
        d_out = self.conv1.backward(d_out, lr)
def train(model, train_X, train_y, epochs=1, lr=0.005):
    for epoch in range(epochs):
        loss = 0
        correct = 0
        for i in range(len(train_X)):
            x, y = train_X[i], train_y[i]
            out = model.forward(x)
            loss += cross_entropy_loss(out, y)
            correct += (np.argmax(out) == y)

            d_out = out
            d_out[y] -= 1  
            model.backward(d_out, lr)

        print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss / len(train_X):.4f} - Accuracy: {correct / len(train_X):.4f}")

model = CNN()
train(model, train_images[:1000], train_labels[:1000], epochs=10, lr=0.005)

correct = 0
for i in range(len(test_images[:200])):
    pred = np.argmax(model.forward(test_images[i]))
    correct += (pred == test_labels[i])

print(f"Test Accuracy: {correct / 200:.4f}")


Training Data: (60000, 28, 28, 1), Labels: (60000,)
Testing Data: (10000, 28, 28, 1), Labels: (10000,)
Epoch 1/10 - Loss: 2.1132 - Accuracy: 0.2250
Epoch 2/10 - Loss: 0.8280 - Accuracy: 0.7440
Epoch 3/10 - Loss: 0.5409 - Accuracy: 0.8450
Epoch 4/10 - Loss: 0.4465 - Accuracy: 0.8700
Epoch 5/10 - Loss: 0.3670 - Accuracy: 0.8900
Epoch 6/10 - Loss: 0.3148 - Accuracy: 0.9100
